# Классификатор на базе YandexGPT Pro

## Ноутбук запускается в Yandex DataSphere

Документация:
- Классификаторы по промту на базе YandexGPT:
https://yandex.cloud/ru/docs/foundation-models/operations/classifier/readymade

- Подготовка инфраструктуры: https://yandex.cloud/ru/docs/tutorials/ml-ai/models-fine-tuning

Для запуска ноутбука необходимо создать секреты с собственным 'FOLDER_ID' и 'API_KEY'


In [1]:
#%pip install --upgrade pip
#%pip install yandexcloud --upgrade
#%pip install yandex-cloud-ml-sdk --upgrade
#%pip install scikit-learn --upgrade

In [48]:
import os
import pandas as pd
import numpy as np
from yandex_cloud_ml_sdk import YCloudML

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.metrics import cohen_kappa_score

Рассматривается Zero-shot подход

In [49]:
df = pd.read_excel('marked_data.xlsx')
df['labels'] = df['labels'] + 1

In [50]:
df_test = df[9600:]

In [51]:
sdk = YCloudML(
        folder_id=os.environ['FOLDER_ID'],
        auth=os.environ['API_KEY'],
    )
    
model = sdk.models.text_classifiers("yandexgpt").configure(
        task_description="определи тип эмоционального окраса комментария",
        labels=["негативный", "нейтральный", "позитивный"],
    )

In [52]:
pred = []
p_neg = []
p_neu = []
p_pos = []
for i in df_test.index:
    request_text = df_test['text'][i]
    result = model.run(request_text)
    pred.append(np.argmax([result[0]['confidence'], result[1]['confidence'], result[2]['confidence']]))
    p_neg.append(result[0]['confidence'])
    p_neu.append(result[1]['confidence'])
    p_pos.append(result[2]['confidence'])

In [53]:
df_test_res0 = df_test.copy()
df_test_res0['predict'] = pred
df_test_res0['Pneg'] = p_neg
df_test_res0['Pneutral'] = p_neu
df_test_res0['Ppos'] = p_pos
df_test_res0['Ppos-Pneg'] = df_test_res0['Ppos'] - df_test_res0['Pneg']
df_test_res0

,post_id,text,labels,predict,Pneg,Pneutral,Ppos,Ppos-Pneg
9600,1153020,"А чего их поддерживать,не умеют ,не могут,пуст...",1,0,9.999990e-01,7.107128e-12,5.833886e-13,-9.999990e-01
9601,3608305,Я давно на них в суд подал. \nГражданский!,1,0,9.999983e-01,7.224434e-07,1.452818e-10,-9.999983e-01
9602,1569787,Северные народы раньше вообще не мылись и толь...,0,0,9.999987e-01,2.699575e-07,2.476530e-09,-9.999987e-01
9603,3651054,у него листок каннабиса в груди?,1,1,1.367024e-03,9.986320e-01,7.128383e-09,-1.367017e-03
9604,3641012,Кто-то правительство развел.🤔,0,1,2.575952e-07,9.999987e-01,3.300642e-08,-2.245888e-07
...,...,...,...,...,...,...,...,...
9995,1139301,"Не, ну это вообще пи.дец...!😡😤😡😤",2,0,9.999990e-01,1.694445e-13,2.553619e-13,-9.999990e-01
9996,5343030,"Чего он сказал-то? Как назвал , оскорбил? Ниче...",0,0,9.999990e-01,3.674400e-09,2.189150e-11,-9.999990e-01
9997,5595571,"Они все , включая и эту НаЕбулину , живут в Пу...",0,0,9.999990e-01,5.180188e-11,1.398110e-12,-9.999990e-01
9998,6590679,"ну так власятм спасибо скажите , это их рук дело",0,0,9.999960e-01,2.948014e-06,1.252779e-08,-9.999960e-01


In [54]:
print('accuracy', accuracy_score(df_test_res0['labels'], df_test_res0['predict']))
print('macro_precision', precision_score(df_test_res0['labels'], df_test_res0['predict'], average='macro'))
print('macro_recall', recall_score(df_test_res0['labels'], df_test_res0['predict'], average='macro'))
print('macro_f1', f1_score(df_test_res0['labels'], df_test_res0['predict'], average='macro'))
print('RMSE for Ppos-Pneg', root_mean_squared_error(df_test_res0['labels']-1, df_test_res0['Ppos-Pneg']))
print('MAE for Ppos-Pneg', mean_absolute_error(df_test_res0['labels']-1, df_test_res0['Ppos-Pneg']))

accuracy 0.5975
macro_precision 0.556299972075476
macro_recall 0.5618665816338942
macro_f1 0.5587642832950068
RMSE for Ppos-Pneg 0.802318324897733
MAE for Ppos-Pneg 0.4816723775110387


In [55]:
print('precision by class', precision_score(df_test_res0['labels'], df_test_res0['predict'], average=None))
print('recall by class', recall_score(df_test_res0['labels'], df_test_res0['predict'], average=None))
print('f1 by class', f1_score(df_test_res0['labels'], df_test_res0['predict'], average=None))

precision by class [0.70103093 0.53731343 0.43055556]
recall by class [0.69035533 0.52554745 0.46969697]
f1 by class [0.69565217 0.53136531 0.44927536]


In [56]:
confusion_matrix(df_test_res0['labels'], df_test_res0['predict'], labels=[0,1,2])

array([[136,  41,  20],
       [ 44,  72,  21],
       [ 14,  21,  31]])

In [57]:
cohen_kappa_score(df_test_res0['labels'], df_test_res0['predict'], labels=None, weights= 'quadratic', sample_weight=None)

0.4143712841523971

In [58]:
df_test_res0.to_excel('YGPT_Zero-shot_9600-9999.xlsx')